Referência da aula: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/lesson/lesson-1-pytorch-basics-and-linear-regression

# **Introdução à regressão linear**

Neste tutorial, discutiremos um dos algoritmos fundamentais do aprendizado de máquina: *Regressão linear*. Criaremos um modelo que prevê o rendimento das safras de maçãs e laranjas (*variáveis-alvo*) observando a temperatura média, a precipitação e a umidade (*variáveis de entrada ou features*) em uma região. Aqui estão os dados de treinamento:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

Em um modelo de regressão linear, cada variável-alvo é estimada como uma soma ponderada das variáveis de entrada, compensada por alguma constante, conhecida como viés:

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visualmente, isso significa que o rendimento das maçãs é uma função linear ou plana da temperatura, da precipitação e da umidade:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

A parte de *aprendizado* da regressão linear consiste em descobrir um conjunto de pesos `w11, w12,... w23, b1 e b2` usando os dados de treinamento para fazer previsões precisas para novos dados. Os pesos _aprendidos_ serão usados para prever a produção de maçãs e laranjas em uma nova região usando a temperatura média, a precipitação e a umidade dessa região.

Vamos _treinar_ nosso modelo ajustando ligeiramente os pesos várias vezes para fazer previsões melhores, usando uma técnica de otimização chamada *gradient descent*. Vamos começar importando o Numpy e o PyTorch.

In [62]:
import numpy as np
import torch

In [63]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [64]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [65]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# **Regressão Linear do Zero**

In [66]:
weight = torch.randn(2,3, requires_grad=True)
bias = torch.randn(2,requires_grad=True)

weight, bias

(tensor([[ 1.8899,  0.7672,  0.8527],
         [-1.0839,  0.0552,  1.3619]], requires_grad=True),
 tensor([ 1.5510, -0.7765], requires_grad=True))

> Definindo o modelo:

> - Inicializando os pesos e os vieses como sendo matriz e vetor, respectivamente  
> - Definindo-os como valores aleatórios pois ainda não é possível saber quais são os valores ideiais para essas features

In [67]:
inputs @ weight.t() + bias

tensor([[227.5801, -17.6408],
        [295.6157,  -7.3921],
        [318.2291,  -8.6863],
        [258.8593, -58.5713],
        [265.2914,  25.0671]], grad_fn=<AddBackward0>)

> O vetor de exemplo da função acima representa a previsão da produção de maçãs e laranjas, respectivamente para cada uma das 5 regiões.

In [68]:
def model(x):
  return x @ weight.t() + bias

In [69]:
predictions = model(inputs)
predictions

tensor([[227.5801, -17.6408],
        [295.6157,  -7.3921],
        [318.2291,  -8.6863],
        [258.8593, -58.5713],
        [265.2914,  25.0671]], grad_fn=<AddBackward0>)

In [70]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [71]:
diff = predictions - targets
torch.sum(diff * diff) / diff.numel()

tensor(25509.4688, grad_fn=<DivBackward0>)

# **Função de Perda (Loss)**

Antes de aprimorarmos nosso modelo, precisamos de uma maneira de avaliar o desempenho do nosso modelo. Podemos comparar as previsões do modelo com os alvos reais usando o seguinte método:

* Calcule a diferença entre as duas matrizes (`predictions` e `targets`).
* Eleve todos os elementos da matriz de diferença ao quadrado para remover os valores negativos.
* Calcular a média dos elementos na matriz resultante.

O resultado é um único número, conhecido como *mean squared error* (MSE).

In [72]:
def mse(preds, targets):
  diff = preds - targets
  return torch.sum(diff * diff) / diff.numel()

In [73]:
loss = mse(predictions, targets)
loss

tensor(25509.4688, grad_fn=<DivBackward0>)

Veja como podemos interpretar o resultado: *Em média, cada elemento da previsão difere da meta real pela raiz quadrada da perda*. E isso é muito ruim, considerando que os números que estamos tentando prever estão na faixa de 50 a 200. O resultado é chamado de *perda* porque indica o quanto o modelo é ruim na previsão das variáveis-alvo. Ele representa a perda de informações no modelo: quanto menor a perda, melhor o modelo.

# **Calcular gradientes**

Com o PyTorch, podemos calcular automaticamente o gradiente ou a derivada da perda em relação aos pesos e vieses porque eles têm `requires_grad` definido como `True`. Veremos como isso é útil daqui a pouco.

In [74]:
loss.backward()

In [75]:
print(weight)
print(weight.grad)

tensor([[ 1.8899,  0.7672,  0.8527],
        [-1.0839,  0.0552,  1.3619]], requires_grad=True)
tensor([[16949.2109, 16568.7324, 10558.5654],
        [-8963.5625, -9504.7051, -5806.9795]])


In [76]:
print(bias)
print(bias.grad)

tensor([ 1.5510, -0.7765], requires_grad=True)
tensor([ 196.9151, -105.4447])


# **Ajuste pesos e preconceitos para reduzir a perda**

A perda (loss) é uma [função quadrática](https://en.wikipedia.org/wiki/Quadratic_function) de nossos pesos e vieses, e nosso objetivo é encontrar o conjunto de pesos onde a perda é menor. Se traçarmos um gráfico da perda em relação a qualquer peso individual ou elemento de tendência, ele se parecerá com a figura mostrada abaixo. Um insight importante do cálculo é que o gradiente indica a taxa de variação da perda, ou seja, a [inclinação](https://en.wikipedia.org/wiki/Slope) da função de perda em relação aos pesos e vieses.

Se um elemento gradiente for **positivo**:

* **aumentar** ligeiramente o valor do elemento de peso **aumentará** a perda
* **diminuir** ligeiramente o valor do elemento de peso **diminuirá** a perda

![gradiente positivo](https://i.imgur.com/WLzJ4xP.png)

Se um elemento gradiente for **negativo**:

* **aumentar** ligeiramente o valor do elemento de peso **diminuirá** a perda
* **diminuir** ligeiramente o valor do elemento de peso **aumentará** a perda

![negativo=gradiente](https://i.imgur.com/dvG2fxU.png)

O aumento ou diminuição da perda pela alteração de um elemento de peso é proporcional ao gradiente da perda em relação a esse elemento. Esta observação forma a base do _algoritmo de otimização de descida do gradiente_ que usaremos para melhorar nosso modelo (_descendo_ ao longo do _gradiente_).

Podemos subtrair de cada elemento de peso uma pequena quantidade proporcional à derivada da perda em relação a esse elemento para reduzir ligeiramente a perda.

In [77]:
weight, weight.grad * 1e-5

(tensor([[ 1.8899,  0.7672,  0.8527],
         [-1.0839,  0.0552,  1.3619]], requires_grad=True),
 tensor([[ 0.1695,  0.1657,  0.1056],
         [-0.0896, -0.0950, -0.0581]]))

In [78]:
with torch.no_grad():
  weight -= weight.grad * 1e-5
  bias -= bias.grad * 1e-5

In [79]:
weight, bias

(tensor([[ 1.7204,  0.6015,  0.7471],
         [-0.9943,  0.1503,  1.4199]], requires_grad=True),
 tensor([ 1.5490, -0.7754], requires_grad=True))

Multiplicamos os gradientes com um número muito pequeno (`10^-5` neste caso) para garantir que não modificaremos muito os pesos. Queremos dar um pequeno passo na direção descendente do gradiente, não um salto gigante. Este número é chamado de *taxa de aprendizagem* do algoritmo.

Usamos `torch.no_grad` para indicar ao PyTorch que não devemos rastrear, calcular ou modificar gradientes ao atualizar os pesos e tendências.

In [80]:
predictions = model(inputs)
loss = mse(predictions, targets)
loss

tensor(17516.9805, grad_fn=<DivBackward0>)

Antes de prosseguir, redefinimos os gradientes para zero invocando o método `.zero_()`. Precisamos fazer isso porque o PyTorch acumula gradientes. Caso contrário, na próxima vez que invocarmos `.backward` na perda, os novos valores de gradiente serão adicionados aos gradientes existentes, o que pode levar a resultados inesperados.

In [81]:
weight.grad.zero_()
bias.grad.zero_()
print(weight.grad)
print(bias.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# **Treine o modelo usando gradiente descendente**

Como visto acima, reduzimos a perda e melhoramos nosso modelo usando o algoritmo de otimização gradiente descendente. Assim, podemos _treinar_ o modelo usando as seguintes etapas:

1. Gere previsões

2. Calcule a perda

3. Calcular gradientes em relação aos pesos e vieses

4. Ajuste os pesos subtraindo uma pequena quantidade proporcional ao gradiente

5. Redefina os gradientes para zero

Vamos implementar o passo a passo acima.

In [82]:
# Gerando predições
predictions = model(inputs)
predictions

tensor([[199.5640,  -2.2312],
        [258.8520,  12.8464],
        [275.1552,  15.2174],
        [230.5379, -43.1918],
        [230.2975,  44.4424]], grad_fn=<AddBackward0>)

In [83]:
# Calculando a perda
loss = mse(predictions, targets)
loss

tensor(17516.9805, grad_fn=<DivBackward0>)

In [84]:
# Calculando os gradientes
loss.backward()
weight.grad, bias.grad

(tensor([[14060.9199, 13476.4492,  8647.9014],
         [-7373.1963, -7797.1304, -4753.0581]]),
 tensor([162.6813, -86.5834]))

Vamos atualizar os pesos e vieses usando os gradientes calculados acima.

In [85]:
# Ajustando os pesos e resetando os gradientes
with torch.no_grad():
  weight -= weight.grad * 1e-5
  bias -= bias.grad * 1e-5
  weight.grad.zero_()
  bias.grad.zero_()

In [86]:
weight, bias

(tensor([[ 1.5798,  0.4667,  0.6606],
         [-0.9205,  0.2283,  1.4675]], requires_grad=True),
 tensor([ 1.5474, -0.7746], requires_grad=True))

Com os novos pesos e vieses, o modelo deveria ter uma perda menor.

In [87]:
# Calculando a perda
predictions = model(inputs)
loss = mse(predictions, targets)
loss

tensor(12126.9658, grad_fn=<DivBackward0>)

Já alcançamos uma redução significativa na perda apenas ajustando ligeiramente os pesos e desvios usando a descida gradiente.

# **Treine para várias épocas**

Para reduzir ainda mais a perda, podemos repetir o processo de ajuste dos pesos e desvios usando os gradientes várias vezes. Cada iteração é chamada de _época_. Vamos treinar o modelo por 100 épocas.

In [88]:
# Treino para 100 épocas
for i in range(100):
  predictions = model(inputs)
  loss = mse(predictions, targets)
  loss.backward()
  with torch.no_grad():
    weight -= weight.grad * 1e-5
    bias -= bias.grad * 1e-5

Verificando que a perda agora é menor:

In [89]:
# Calculando a perda
predictions = model(inputs)
loss = mse(predictions, targets)
loss

tensor(2201.6675, grad_fn=<DivBackward0>)

A perda é agora muito inferior ao seu valor inicial. Vejamos as previsões do modelo e compare-as com as metas.

In [90]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [91]:
predictions

tensor([[101.3021,  45.5736],
        [143.2536,  61.7206],
        [192.6663, 104.8113],
        [ 54.0299,  15.9840],
        [168.6469,  74.8305]], grad_fn=<AddBackward0>)

As previsões estão agora bastante próximas das variáveis-alvo. Podemos obter resultados ainda melhores treinando por mais algumas épocas.

# **Regressão linear usando PyTorch integrado**

Implementamos regressão linear e modelo de gradiente descendente usando algumas operações básicas de tensor. No entanto, como esse é um padrão comum no aprendizado profundo, o PyTorch fornece várias funções e classes integradas para facilitar a criação e o treinamento de modelos com apenas algumas linhas de código.

Vamos começar importando o pacote `torch.nn` do PyTorch, que contém classes utilitárias para construção de redes neurais.

In [92]:
import torch.nn as nn

In [93]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [94]:
inputs.shape

torch.Size([15, 3])

# **Dataset and DataLoader**

In [95]:
from torch.utils.data import TensorDataset

In [96]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

O `TensorDataset` nos permite acessar uma pequena seção dos dados de treinamento usando a notação de indexação de matriz (`[0:3]` no código acima). Ele retorna uma tupla com dois elementos. O primeiro elemento contém as variáveis de entrada para as linhas selecionadas e o segundo contém os alvos.

Também criaremos um `DataLoader`, que pode dividir os dados em lotes de tamanho predefinido durante o treinamento. Ele também fornece outros utilitários, como embaralhamento e amostragem aleatória dos dados.

In [97]:
from torch.utils.data import DataLoader

In [98]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [99]:
# Carregando os três lotes de dados do conjunto
for xb, yb in train_dl:
  print('Lote')
  print(xb)
  print(yb)

Lote
tensor([[ 88., 134.,  59.],
        [ 91.,  87.,  65.],
        [103.,  43.,  36.],
        [ 73.,  66.,  44.],
        [102.,  43.,  37.]])
tensor([[118., 132.],
        [ 80., 102.],
        [ 20.,  38.],
        [ 57.,  69.],
        [ 22.,  37.]])
Lote
tensor([[ 73.,  67.,  43.],
        [ 92.,  87.,  64.],
        [ 69.,  96.,  70.],
        [ 87., 135.,  57.],
        [101.,  44.,  37.]])
tensor([[ 56.,  70.],
        [ 82., 100.],
        [103., 119.],
        [118., 134.],
        [ 21.,  38.]])
Lote
tensor([[ 68.,  96.,  71.],
        [ 68.,  97.,  70.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 74.,  66.,  43.]])
tensor([[104., 118.],
        [102., 120.],
        [ 81., 101.],
        [119., 133.],
        [ 57.,  69.]])


# **Classe nn.Linear**

In [100]:
# nn.Linear(input_size, num_classes)
# input_size = humidade, temperatura, chuva
# num_classes = rendimento de maçãs e laranjas
model = nn.Linear(3,2)
# weight e bias aqui são parâmetros padrão
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1950,  0.4197,  0.2940],
        [ 0.4141, -0.1025, -0.4840]], requires_grad=True)
Parameter containing:
tensor([-0.2339, -0.0552], requires_grad=True)


In [102]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1950,  0.4197,  0.2940],
         [ 0.4141, -0.1025, -0.4840]], requires_grad=True),
 Parameter containing:
 tensor([-0.2339, -0.0552], requires_grad=True)]

In [103]:
preds = model(inputs)
preds

tensor([[ 26.2959,   2.4900],
        [ 37.7744,  -2.3740],
        [ 56.0985,  -5.8421],
        [  8.8035,  19.8622],
        [ 47.1860, -15.2074],
        [ 25.6812,   3.0066],
        [ 37.6486,  -2.7554],
        [ 56.1975,  -5.9121],
        [  9.4182,  19.3456],
        [ 47.6749, -16.1055],
        [ 26.1702,   2.1085],
        [ 37.1596,  -1.8574],
        [ 56.2243,  -5.4606],
        [  8.3145,  20.7603],
        [ 47.8007, -15.7240]], grad_fn=<AddmmBackward0>)

# **Função de Perda (Loss)**

In [110]:
import torch.nn.functional as F

In [111]:
loss_fn = F.mse_loss

In [112]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6275.7642, grad_fn=<MseLossBackward0>)


# **Otimizador**

Em vez de manipular manualmente os pesos e as tendências do modelo usando gradientes, podemos usar o otimizador `optim.SGD`. SGD é a abreviação de “stochastic gradient descent” (descida de gradiente estocástica). O termo _stochastic_ indica que as amostras são selecionadas em lotes aleatórios em vez de em um único grupo.

In [113]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Observe que `model.parameters()` é passado como um argumento para `optim.SGD` para que o otimizador saiba quais matrizes devem ser modificadas durante a etapa de atualização. Além disso, podemos especificar uma taxa de aprendizado que controla o valor pelo qual os parâmetros são modificados.

# **Treinando o Modelo**

Agora estamos prontos para treinar o modelo. Seguiremos o mesmo processo para implementar a descida de gradiente:

1. Gerar previsões

2. Calcular a perda

3. Calcule gradientes com relação aos pesos e vieses

4. Ajuste os pesos subtraindo uma pequena quantidade proporcional ao gradiente

5. Redefinir os gradientes para zero

A única mudança é que trabalharemos com lotes de dados em vez de processar todos os dados de treinamento em cada iteração. Vamos definir uma função utilitária `fit` que treina o modelo para um determinado número de épocas.

In [114]:
# Função utilitária para treinar o modelo
def fit(num_epochs, model, loss_fn, opt, train_dl):

  # Repetindo o processo para o número de épocas determinado
  for epoch in range(num_epochs):

    # Treinando com os lotes (batches) de dados
    for xb, yb in train_dl:

      # Gerando predições
      predictions = model(xb)

      # Calculando a perda
      loss = loss_fn(predictions, yb)

      # Calculando os gradientes
      loss.backward()

      # Atualizando os parâmetros utilizando os gradientes
      opt.step()

      # Resetando os gradientes para zero
      opt.zero_grad()

      if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Alguns aspectos a serem observados acima:

* Usamos o carregador de dados definido anteriormente para obter lotes de dados para cada iteração.

* Em vez de atualizar os parâmetros (pesos e bias) manualmente, usamos `opt.step` para realizar a atualização e `opt.zero_grad` para redefinir os gradientes para zero.

* Também adicionamos uma instrução de registro que imprime a perda do último lote de dados a cada 10 épocas para acompanhar o progresso do treinamento. O `loss.item` retorna o valor real armazenado no tensor de perda.

Vamos treinar o modelo para 100 épocas.

In [115]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 377.4854
Epoch [10/100], Loss: 358.4431
Epoch [10/100], Loss: 629.1356
Epoch [20/100], Loss: 309.6006
Epoch [20/100], Loss: 393.0907
Epoch [20/100], Loss: 239.9509
Epoch [30/100], Loss: 155.7235
Epoch [30/100], Loss: 226.3564
Epoch [30/100], Loss: 298.7207
Epoch [40/100], Loss: 168.9968
Epoch [40/100], Loss: 170.6913
Epoch [40/100], Loss: 161.3027
Epoch [50/100], Loss: 54.1205
Epoch [50/100], Loss: 159.2005
Epoch [50/100], Loss: 177.6234
Epoch [60/100], Loss: 149.5891
Epoch [60/100], Loss: 97.9553
Epoch [60/100], Loss: 49.1023
Epoch [70/100], Loss: 79.7201
Epoch [70/100], Loss: 79.8000
Epoch [70/100], Loss: 73.8776
Epoch [80/100], Loss: 57.8007
Epoch [80/100], Loss: 45.1989
Epoch [80/100], Loss: 92.0613
Epoch [90/100], Loss: 67.2296
Epoch [90/100], Loss: 34.1094
Epoch [90/100], Loss: 60.7971
Epoch [100/100], Loss: 34.6929
Epoch [100/100], Loss: 57.3966
Epoch [100/100], Loss: 49.2684


In [116]:
preds = model(inputs)
preds

tensor([[ 57.5618,  72.4374],
        [ 81.1075,  96.2386],
        [119.1783, 139.6667],
        [ 24.5695,  48.8064],
        [ 98.0259, 104.5331],
        [ 56.3820,  71.4879],
        [ 80.7590,  95.4271],
        [119.3867, 139.8368],
        [ 25.7493,  49.7559],
        [ 98.8572, 104.6710],
        [ 57.2133,  71.6258],
        [ 79.9276,  95.2892],
        [119.5268, 140.4783],
        [ 23.7381,  48.6685],
        [ 99.2057, 105.4826]], grad_fn=<AddmmBackward0>)

In [117]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

De fato, as previsões estão bem próximas de nossas metas. Treinamos um modelo razoavelmente bom para prever o rendimento das safras de maçãs e laranjas, observando a temperatura média, a precipitação e a umidade em uma região. Podemos usá-lo para fazer previsões de rendimentos de colheitas para novas regiões passando um lote contendo uma única linha de entrada.

In [118]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.9853, 68.7613]], grad_fn=<AddmmBackward0>)